In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Parameters

### Fetching Parameters and Creating Variables

In [0]:
# # Key Cols List
# key_cols = "['flight_id']"
# key_cols_list = eval(key_cols)

# # Catalog
# catalog = "workspace"

# #CDC Column
# cdc_col = "modifiedDate"

# #Back-Dated Refresh
# back_dated_refresh = ""

# #Source Object
# source_object = "silver_flights"

# #Source Schema
# source_schema = "silver"

# #Target Schema
# target_schema = "gold"

# #Target Object
# target_object = "DimFlights"

# # Surrogate Key Column
# surrogate_key = "DimFlightKey"



In [0]:
# # Key Cols List
# key_cols = "['airport_id']"
# key_cols_list = eval(key_cols)

# # Catalog
# catalog = "workspace"

# #CDC Column
# cdc_col = "modifiedDate"

# #Back-Dated Refresh
# back_dated_refresh = ""

# #Source Object
# source_object = "silver_airports"

# #Source Schema
# source_schema = "silver"

# #Target Schema
# target_schema = "gold"

# #Target Object
# target_object = "DimAirports"

# # Surrogate Key Column
# surrogate_key = "DimAirportKey"

In [0]:
# Key Cols List
key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)

# Catalog
catalog = "workspace"

#CDC Column
cdc_col = "modifiedDate"

#Back-Dated Refresh
back_dated_refresh = ""

#Source Object
source_object = "silver_passengers"

#Source Schema
source_schema = "silver"

#Target Schema
target_schema = "gold"

#Target Object
target_object = "DimPassengers"

# Surrogate Key Column
surrogate_key = "DimPassengerKey"

In [0]:
key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)
key_cols_list

['passenger_id']

### **Incremental Data Ingestion**

### Last Load Date


In [0]:
# No Back Dated Refresh 
if len(back_dated_refresh) == 0:
    
    # If Table exists in the Destination 
    if spark.catalog.tableExists(f"{target_schema}.{target_object}"):
        last_load = spark.sql(f"SELECT max({cdc_col}) from {catalog}.{target_schema}.{target_object}").collect()[0][0]
   
    # set a default old date
    else:
        last_load = "1900-01-01 00:00:00"

# Yes Back Dated Refresh
else:
    last_load = back_dated_refresh

# Test Last Load
last_load



datetime.datetime(2025, 7, 7, 2, 25, 1, 262000)

In [0]:

# filters only new or updated records in the Silver layer
df_src = spark.sql(f"SELECT * from {source_schema}.{source_object} WHERE {cdc_col} >'{last_load}'")


In [0]:
df_src.display()

passenger_id,name,gender,nationality,modifiedDate
P0223,Nicholas Gomez,Female,Cook Islands,2025-07-09T09:26:49.232Z
P0224,Jason Jensen,Female,Rwanda,2025-07-09T09:26:49.232Z
P0222,Maria Taylor,Male,Lao People's Democratic Republic,2025-07-09T09:26:49.232Z
P0225,William Lopez,Male,Heard Island and McDonald Islands,2025-07-09T09:26:49.232Z
P0221,Amy Welch,Male,Croatia,2025-07-09T09:26:49.232Z
P0032,Daniel Douglas,Male,Singapore,2025-07-09T09:26:49.232Z
P0106,Joshua Shepherd Jr.,Female,Netherlands,2025-07-09T09:26:49.232Z
P0108,Brian Anderson,Female,Eritrea,2025-07-09T09:26:49.232Z
P0154,Angel Thompson,Female,Gambia,2025-07-09T09:26:49.232Z
P0149,Katherine Bowen DVM,Male,Mexico,2025-07-09T09:26:49.232Z


## **Old Vs New Records**

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    #Key Columns String
    key_cols_string_incremental = ','.join(key_cols_list)

    df_trg = spark.sql(f"SELECT {key_cols_string_incremental}, {surrogate_key}, create_date, update_date FROM {catalog}.{target_schema}.{target_object}")
else:

    # Key Column string for initial
    key_cols_string_init = [f"'' AS {i}" for i in key_cols_list]
    key_cols_string_init = ','.join(key_cols_string_init)
    
    df_trg = spark.sql(f"SELECT {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key}, cast('1900-01-01 00:00:00' AS timestamp) AS create_date, cast('1900-01-01 00:00:00' AS timestamp) AS update_date FROM (SELECT 1) WHERE 1=0")

In [0]:
df_trg.display()

passenger_id,DimPassengerKey,create_date,update_date
P0002,1,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0004,2,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0007,3,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0014,4,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0023,5,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0033,6,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0049,7,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0080,8,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0120,9,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0134,10,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z


### JOIN CONDITION

In [0]:

join_condition ='AND'.join([f"src.{i} = trg.{i}" for i in key_cols_list])
join_condition

'src.passenger_id = trg.passenger_id'

In [0]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

df_join = spark.sql(f"""
            SELECT src.*,
                   trg.{surrogate_key},
                   trg.create_date,
                   trg.update_date
            FROM src
            LEFT JOIN trg
            ON {join_condition}
            """
)

In [0]:
df_join.display()

passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0155,Robert Singleton,Female,Northern Mariana Islands,2025-07-09T09:26:49.232Z,82,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0106,Joshua Shepherd Jr.,Female,Netherlands,2025-07-09T09:26:49.232Z,197,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0032,Daniel Douglas,Male,Singapore,2025-07-09T09:26:49.232Z,46,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0154,Angel Thompson,Female,Gambia,2025-07-09T09:26:49.232Z,110,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0196,Monica Dillon,Female,Belarus,2025-07-09T09:26:49.232Z,86,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0049,Justin Thomas,Female,Tokelau,2025-07-09T09:26:49.232Z,7,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0108,Brian Anderson,Female,Eritrea,2025-07-09T09:26:49.232Z,36,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0113,Brian Richardson,Male,Dominican Republic,2025-07-09T09:26:49.232Z,50,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0158,Jamie Mcneil,Male,Burundi,2025-07-09T09:26:49.232Z,76,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0149,Katherine Bowen DVM,Male,Mexico,2025-07-09T09:26:49.232Z,53,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z


In [0]:
df_old = df_join.filter(col(f'{surrogate_key}').isNotNull())
df_new = df_join.filter(col(f'{surrogate_key}').isNull())


In [0]:
df_old.display()
df_new.display()

passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0049,Justin Thomas,Female,Tokelau,2025-07-09T09:26:49.232Z,7,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0108,Brian Anderson,Female,Eritrea,2025-07-09T09:26:49.232Z,36,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0032,Daniel Douglas,Male,Singapore,2025-07-09T09:26:49.232Z,46,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0113,Brian Richardson,Male,Dominican Republic,2025-07-09T09:26:49.232Z,50,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0149,Katherine Bowen DVM,Male,Mexico,2025-07-09T09:26:49.232Z,53,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0158,Jamie Mcneil,Male,Burundi,2025-07-09T09:26:49.232Z,76,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0155,Robert Singleton,Female,Northern Mariana Islands,2025-07-09T09:26:49.232Z,82,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0196,Monica Dillon,Female,Belarus,2025-07-09T09:26:49.232Z,86,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0154,Angel Thompson,Female,Gambia,2025-07-09T09:26:49.232Z,110,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z
P0106,Joshua Shepherd Jr.,Female,Netherlands,2025-07-09T09:26:49.232Z,197,2025-07-09T09:17:40.228Z,2025-07-09T09:17:40.228Z


passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0225,William Lopez,Male,Heard Island and McDonald Islands,2025-07-09T09:26:49.232Z,null,null,null
P0224,Jason Jensen,Female,Rwanda,2025-07-09T09:26:49.232Z,null,null,null
P0223,Nicholas Gomez,Female,Cook Islands,2025-07-09T09:26:49.232Z,null,null,null
P0222,Maria Taylor,Male,Lao People's Democratic Republic,2025-07-09T09:26:49.232Z,null,null,null
P0221,Amy Welch,Male,Croatia,2025-07-09T09:26:49.232Z,null,null,null


##**ENRICHING DFS**

### Preparing DF_OLD

In [0]:
df_old_enr = df_old.withColumn('update_date',current_timestamp())

In [0]:
df_old_enr.display()

passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0049,Justin Thomas,Female,Tokelau,2025-07-09T09:26:49.232Z,7,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0108,Brian Anderson,Female,Eritrea,2025-07-09T09:26:49.232Z,36,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0032,Daniel Douglas,Male,Singapore,2025-07-09T09:26:49.232Z,46,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0113,Brian Richardson,Male,Dominican Republic,2025-07-09T09:26:49.232Z,50,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0149,Katherine Bowen DVM,Male,Mexico,2025-07-09T09:26:49.232Z,53,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0158,Jamie Mcneil,Male,Burundi,2025-07-09T09:26:49.232Z,76,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0155,Robert Singleton,Female,Northern Mariana Islands,2025-07-09T09:26:49.232Z,82,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0196,Monica Dillon,Female,Belarus,2025-07-09T09:26:49.232Z,86,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0154,Angel Thompson,Female,Gambia,2025-07-09T09:26:49.232Z,110,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z
P0106,Joshua Shepherd Jr.,Female,Netherlands,2025-07-09T09:26:49.232Z,197,2025-07-09T09:17:40.228Z,2025-07-09T09:33:54.166Z


### Preparing DF_NEW

In [0]:
df_new.display()


passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0225,William Lopez,Male,Heard Island and McDonald Islands,2025-07-09T09:26:49.232Z,null,null,null
P0224,Jason Jensen,Female,Rwanda,2025-07-09T09:26:49.232Z,null,null,null
P0223,Nicholas Gomez,Female,Cook Islands,2025-07-09T09:26:49.232Z,null,null,null
P0222,Maria Taylor,Male,Lao People's Democratic Republic,2025-07-09T09:26:49.232Z,null,null,null
P0221,Amy Welch,Male,Croatia,2025-07-09T09:26:49.232Z,null,null,null


In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
     max_surrogate_key = spark.sql(f""" 
          SELECT max({surrogate_key}) AS max_surrogate_key
          FROM {catalog}.{target_schema}.{target_object}
          """).collect()[0][0]
     df_new_enr = df_new.withColumn(f'{surrogate_key}',lit(max_surrogate_key) + lit(1) + monotonically_increasing_id())\
                   .withColumn('create_date',current_timestamp())\
                   .withColumn('update_date',current_timestamp())


else:
    max_surrogate_key = 0
    df_new_enr = df_new.withColumn(f'{surrogate_key}',lit(max_surrogate_key) + lit(1) + monotonically_increasing_id())\
                   .withColumn('create_date',current_timestamp())\
                   .withColumn('update_date',current_timestamp())

   
    

In [0]:
max_surrogate_key

220

In [0]:
df_new_enr.display()

passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0225,William Lopez,Male,Heard Island and McDonald Islands,2025-07-09T09:26:49.232Z,221,2025-07-09T09:33:57.522Z,2025-07-09T09:33:57.522Z
P0224,Jason Jensen,Female,Rwanda,2025-07-09T09:26:49.232Z,222,2025-07-09T09:33:57.522Z,2025-07-09T09:33:57.522Z
P0223,Nicholas Gomez,Female,Cook Islands,2025-07-09T09:26:49.232Z,223,2025-07-09T09:33:57.522Z,2025-07-09T09:33:57.522Z
P0222,Maria Taylor,Male,Lao People's Democratic Republic,2025-07-09T09:26:49.232Z,224,2025-07-09T09:33:57.522Z,2025-07-09T09:33:57.522Z
P0221,Amy Welch,Male,Croatia,2025-07-09T09:26:49.232Z,225,2025-07-09T09:33:57.522Z,2025-07-09T09:33:57.522Z


##**UNIONING OLD AND NEW RECORDS**

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)


In [0]:
df_union.display()

passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0049,Justin Thomas,Female,Tokelau,2025-07-09T09:26:49.232Z,7,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0108,Brian Anderson,Female,Eritrea,2025-07-09T09:26:49.232Z,36,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0032,Daniel Douglas,Male,Singapore,2025-07-09T09:26:49.232Z,46,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0113,Brian Richardson,Male,Dominican Republic,2025-07-09T09:26:49.232Z,50,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0149,Katherine Bowen DVM,Male,Mexico,2025-07-09T09:26:49.232Z,53,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0158,Jamie Mcneil,Male,Burundi,2025-07-09T09:26:49.232Z,76,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0155,Robert Singleton,Female,Northern Mariana Islands,2025-07-09T09:26:49.232Z,82,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0196,Monica Dillon,Female,Belarus,2025-07-09T09:26:49.232Z,86,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0154,Angel Thompson,Female,Gambia,2025-07-09T09:26:49.232Z,110,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z
P0106,Joshua Shepherd Jr.,Female,Netherlands,2025-07-09T09:26:49.232Z,197,2025-07-09T09:17:40.228Z,2025-07-09T09:33:59.167Z


##**UPSERT**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark,f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_union.alias("src"),f"trg.{surrogate_key} = src.{surrogate_key}")\
                        .whenMatchedUpdateAll(condition = f"src.{cdc_col}>=trg.{cdc_col}")\
                        .whenNotMatchedInsertAll()\
                        .execute()
else:
    df_union.write.format('delta')\
                  .mode('append')\
                  .saveAsTable(f"{catalog}.{target_schema}.{target_object}")


In [0]:
%sql
select * from gold.dimPassengers where passenger_id = 'P0049'

passenger_id,name,gender,nationality,modifiedDate,DimPassengerKey,create_date,update_date
P0049,Justin Thomas,Female,Tokelau,2025-07-09T09:26:49.232Z,7,2025-07-09T09:17:40.228Z,2025-07-09T09:34:01.476Z
